In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '7'

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms

import matplotlib.pyplot as plt
import cv2
from PIL import Image

%load_ext autoreload
%autoreload 2

In [7]:
import sys
sys.path.append('../')
sys.path.append('../core')

from raft import RAFT
from train import get_args, load_model
import datasets as raft_datasets
import teachers
import utils.flow_viz as flow_viz
import matplotlib
def flow_to_rgb(vec, flow_mag_range=None, white_bg=False):
    height, width = vec.shape[:2]
    scaling = 50. / (height**2 + width**2)**0.5
    direction = (np.arctan2(vec[..., 0], vec[..., 1]) + np.pi) / (2 * np.pi)
    norm = np.linalg.norm(vec, axis=-1)
    if flow_mag_range is None:
        flow_mag_range = norm.min(), norm.max()
        magnitude = np.clip((norm - flow_mag_range[0]) * scaling, 0., 1.)
    if white_bg == True:
        value = np.ones_like(direction)
        hsv = np.stack([direction, magnitude, saturation], axis=-1)
    else:
        saturation = np.ones_like(direction)
        hsv = np.stack([direction, saturation , magnitude], axis=-1)
        rgb = matplotlib.colors.hsv_to_rgb(hsv)
    return rgb

/home/dbear/tenv/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
import dorsalventral.models.targets as targets
import dorsalventral.vis_utils as vis_utils
import dorsalventral.data.utils as data_utils
import dorsalventral.models.fire_propagation as fprop
import dorsalventral.models.layer_utils as layer_utils
import dorsalventral.models.segmentation.competition as competition
import dorsalventral.models.losses as losses
import kornia

/home/dbear/tenv/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
tdw_dataset = raft_datasets.TdwFlowDataset(
#     root='/data5/dbear/tdw_datasets/playroom_large_v3copy/',
    root='/mnt/fs5/dbear/tdw_datasets/playroom_large_v3/',
#     dataset_names=['model_split_4'],    
    dataset_names=['model_split_%d' % split for split in range(5)],
    split='validation',
    filepattern="*",
#     test_filepattern="0*[0-4]",
    test_filepattern="*9",
    min_start_frame=5,
    max_start_frame=5,
    scale_to_pixels=True,
    get_gt_flow=True,
    get_backward_frame=True
)
print(len(tdw_dataset))
print(tdw_dataset.is_test)


data_dir = '/mnt/fs6/honglinc/dataset/tensorflow_datasets/movi_d/256x256/1.0.0'
movi_dataset = raft_datasets.MoviFlowDataset(
    root=data_dir,
    split='validation',
    sequence_length=2
)
print(len(movi_dataset))

/home/dbear/tenv/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


625
True


/home/dbear/tenv/lib/python3.7/site-packages/tensorflow/python/ops/numpy_ops/np_random.py:95: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def randint(low, high=None, size=None, dtype=onp.int):  # pylint: disable=missing-function-docstring


250


In [9]:
# motion_paths = !ls ../checkpoints/[0-9]*motion*dtarg*nthr05*fullplay*pretrained*
# motion_paths = !ls ../checkpoints/[0-9]*motion*tdw*nthr0-cthr025*

# motion_paths = !ls ../checkpoints/[0-9]*motion*movi_d*bs2*nthr0-*gs1*
motion_paths = !ls ../checkpoints/[0-9]*motion*rnd1*movi_d*bs2*gs1*

motion_paths = sorted(motion_paths, key=lambda s: int(s.split('/')[-1].split('_')[0]))
# !ls ../checkpoints/*motion*imtarg*thr0*
for i,p in enumerate(motion_paths):
    print(i, p)

/home/dbear/tenv/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0 ../checkpoints/2500_motion-rnd1-movi_d-bs2-large-mt05-bt05-flit24-gs1-pretrained-0.pth
1 ../checkpoints/5000_motion-rnd1-movi_d-bs2-large-mt05-bt05-flit24-gs1-pretrained-0.pth
2 ../checkpoints/7500_motion-rnd1-movi_d-bs2-large-mt05-bt05-flit24-gs1-pretrained-0.pth
3 ../checkpoints/10000_motion-rnd1-movi_d-bs2-large-mt05-bt05-flit24-gs1-pretrained-0.pth
4 ../checkpoints/12500_motion-rnd1-movi_d-bs2-large-mt05-bt05-flit24-gs1-pretrained-0.pth
5 ../checkpoints/15000_motion-rnd1-movi_d-bs2-large-mt05-bt05-flit24-gs1-pretrained-0.pth
6 ../checkpoints/17500_motion-rnd1-movi_d-bs2-large-mt05-bt05-flit24-gs1-pretrained-0.pth
7 ../checkpoints/20000_motion-rnd1-movi_d-bs2-large-mt05-bt05-flit24-gs1-pretrained-0.pth
8 ../checkpoints/22500_motion-rnd1-movi_d-bs2-large-mt05-bt05-flit24-gs1-pretrained-0.pth
9 ../checkpoints/25000_motion-rnd1-movi_d-bs2-large-mt05-bt05-flit24-gs1-pretrained-0.pth
10 ../checkpoints/27500_motion-rnd1-movi_d-bs2-large-mt05-bt05-flit24-gs1-pretrained-0.pth
11 ../checkp

In [10]:
# boundary_paths = !ls ../checkpoints/[0-9]*boundary*rnd1*dtarg*fullplay*pretrained*

# boundary_paths = !ls ../checkpoints/[0-9]*boundaryStaticReg*rnd0*cthr075*fullplay*rerun*
# boundary_paths = !ls ../checkpoints/[0-9]*boundaryMotionReg*rnd0*small*nthr0*cthr075*fullplay*
# boundary_paths = !ls ../checkpoints/[0-9]*boundaryMotionReg*rnd1*tdw*small*

# boundary_paths = !ls ../checkpoints/[0-9]*boundaryStatic*rnd0*movi_d*nthr0-*gs1*
# boundary_paths = !ls ../checkpoints/100000*boundaryMotion*rnd0*movi_d*nthr0-*gs1*

boundary_paths = !ls ../checkpoints/[0-9]*boundaryMotion*rnd1*movi_d*gs1*


boundary_paths = sorted(boundary_paths, key=lambda s: int(s.split('/')[-1].split('_')[0]))
for i,p in enumerate(boundary_paths):
    print(i, p)

/home/dbear/tenv/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0 ../checkpoints/2500_boundaryMotionReg-rnd1-movi_d-bs2-small-mt05-bt05-flit24-gs1-pretrained-0.pth
1 ../checkpoints/5000_boundaryMotionReg-rnd1-movi_d-bs2-small-mt05-bt05-flit24-gs1-pretrained-0.pth
2 ../checkpoints/7500_boundaryMotionReg-rnd1-movi_d-bs2-small-mt05-bt05-flit24-gs1-pretrained-0.pth
3 ../checkpoints/10000_boundaryMotionReg-rnd1-movi_d-bs2-small-mt05-bt05-flit24-gs1-pretrained-0.pth
4 ../checkpoints/12500_boundaryMotionReg-rnd1-movi_d-bs2-small-mt05-bt05-flit24-gs1-pretrained-0.pth
5 ../checkpoints/15000_boundaryMotionReg-rnd1-movi_d-bs2-small-mt05-bt05-flit24-gs1-pretrained-0.pth
6 ../checkpoints/17500_boundaryMotionReg-rnd1-movi_d-bs2-small-mt05-bt05-flit24-gs1-pretrained-0.pth
7 ../checkpoints/20000_boundaryMotionReg-rnd1-movi_d-bs2-small-mt05-bt05-flit24-gs1-pretrained-0.pth
8 ../checkpoints/22500_boundaryMotionReg-rnd1-movi_d-bs2-small-mt05-bt05-flit24-gs1-pretrained-0.pth
9 ../checkpoints/25000_boundaryMotionReg-rnd1-movi_d-bs2-small-mt05-bt05-flit24-gs1-pretrained

In [11]:
# flow_paths = !ls ../checkpoints/[0-9]*flowBoundary*bmt*movi_d*gs1*
# flow_paths = !ls ../checkpoints/[0-9]*flowBoundary*rnd0*movi_d*gs1*
flow_paths = !ls ../checkpoints/[0-9]*flowBoundary*rnd1*movi_d*gs1*

# flow_paths = !ls ../checkpoints/[0-9]*flowBoundary*tdw*

flow_paths = sorted(flow_paths, key=lambda s: int(s.split('/')[-1].split('_')[0]))
# !ls ../checkpoints/*motion*imtarg*thr0*
for i,p in enumerate(flow_paths):
    print(i, p)

/home/dbear/tenv/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0 ../checkpoints/2500_flowBoundary-rnd1-movi_d-bs2-large-mt05-bt01-flit24-gs1-pretrained-0.pth
1 ../checkpoints/2500_flowBoundary-rnd1-movi_d-bs2-large-mt05-bt01-gs1-pretrained-0.pth
2 ../checkpoints/2500_flowBoundary-rnd1-movi_d-bs2-large-mt05-bt01-gs1-scratch-0.pth
3 ../checkpoints/5000_flowBoundary-rnd1-movi_d-bs2-large-mt05-bt01-flit24-gs1-pretrained-0.pth
4 ../checkpoints/5000_flowBoundary-rnd1-movi_d-bs2-large-mt05-bt01-gs1-pretrained-0.pth
5 ../checkpoints/5000_flowBoundary-rnd1-movi_d-bs2-large-mt05-bt01-gs1-scratch-0.pth
6 ../checkpoints/7500_flowBoundary-rnd1-movi_d-bs2-large-mt05-bt01-flit24-gs1-pretrained-0.pth
7 ../checkpoints/7500_flowBoundary-rnd1-movi_d-bs2-large-mt05-bt01-gs1-scratch-0.pth
8 ../checkpoints/10000_flowBoundary-rnd1-movi_d-bs2-large-mt05-bt01-flit24-gs1-pretrained-0.pth
9 ../checkpoints/10000_flowBoundary-rnd1-movi_d-bs2-large-mt05-bt01-gs1-scratch-0.pth
10 ../checkpoints/12500_flowBoundary-rnd1-movi_d-bs2-large-mt05-bt01-flit24-gs1-pretrained-0.pth
11 ..

In [14]:
m_idx = -1
b_idx = -1
f_idx = -1
m_path = motion_paths[m_idx]
b_path = boundary_paths[b_idx]
f_path = flow_paths[f_idx]

TargetFunc = teachers.MotionToStaticTeacher(
    student_model_type='motion',
    downsample_factor=2,
    spatial_resolution=3,
    motion_resolution=3,
    target_from_motion=False,
    return_intermediates=True,
    build_flow_target=True,
    motion_path=m_path,
    boundary_path=b_path,
    flow_path=f_path,
    motion_model_params={
        'small': 'small' in m_path,
        'gate_stride': 2 if ('gs1' not in m_path) else 1
    },
    boundary_model_params={
        'small': 'small' in b_path,
        'static_input': 'Static' in b_path,
        'orientation_type': 'regression',
        'gate_stride': 2 if ('gs1' not in b_path) else 1
    },
).cuda()
TargetFunc.eval()

/home/dbear/tenv/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


<All keys matched successfully> MotionClassifier ../checkpoints/70000_motion-rnd1-movi_d-bs2-large-mt05-bt05-flit24-gs1-pretrained-0.pth
<All keys matched successfully> BoundaryClassifier ../checkpoints/70000_boundaryMotionReg-rnd1-movi_d-bs2-small-mt05-bt05-flit24-gs1-pretrained-0.pth
<All keys matched successfully> RAFT ../checkpoints/62500_flowBoundary-rnd1-movi_d-bs2-large-mt05-bt01-flit24-gs1-pretrained-0.pth


MotionToStaticTeacher(
  (motion_model): MotionClassifier(
    (fnet): BasicEncoder(
      (norm1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
      (relu1): ReLU(inplace=True)
      (layer1): Sequential(
        (0): ResidualBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (relu): ReLU(inplace=True)
          (norm1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
          (norm2): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        )
        (1): ResidualBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (relu): ReLU

In [13]:
SKP = fprop.SpacetimeLocalKProp(
    num_iters=10
).cuda()

/home/dbear/tenv/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [17]:
ex = 0
img1, img2, flow_gt, segments = [x[None].cuda() for x in movi_dataset[ex]]

/home/dbear/tenv/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


movie, frame b'5397' 21


In [23]:
TargetFunc.student_model_type = None
TargetFunc.target_model.target_type = 'motion'
TargetFunc.return_intermediates = False
fwd = TargetFunc(img1, img2)

/home/dbear/tenv/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


RuntimeError: CUDA out of memory. Tried to allocate 32.00 MiB (GPU 0; 11.91 GiB total capacity; 10.87 GiB already allocated; 4.38 MiB free; 11.11 GiB reserved in total by PyTorch)

In [20]:
fwd.keys()

/home/dbear/tenv/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


AttributeError: 'Tensor' object has no attribute 'keys'